# CMA-ES Optimization for Multiple Problems

This notebook runs CMA-ES on 18 problems (shifted and unshifted, 60D and low-dimensional), 100 times each, saving the best fitness values to files in `EARS comparison/Algorithm results`. Parameters:
- **Max evaluations**: 15,000
- **Population size (lambda)**: 30
- **Initial standard deviation (sigma)**: 0.5
- **Function tolerance**: 1e-8
- **Runs per problem**: 100

**Notes on Initialization:**
- The initial search point (`x0`) is randomly generated within the bounds.

In [4]:
import cma
import numpy as np
from pathlib import Path
import time

# Parameters
number_of_runs = 100
max_evaluations = 15000
n_dimensions = 60
pop_size = 30  # (lambda)
sigma0 = 0.5  # Initial standard deviation


shift_sphere = np.array([
    45.00458616074768, -50.34195790924958, 16.656571994777153, -70.66157384610746, 77.87240204480926,
    41.15636478376551, 49.128266155105166, -60.307473748449034, 33.10423742373693, 0.6236286355758835,
    27.460586290936902, -36.02013445020688, -40.00877054048374, -64.66271141874185, 68.90566619452034,
    62.70237277088032, -4.074486986783384, -1.317873518733137, -55.54285608300457, -47.34319489184719,
    12.507912147062243, -40.535720288582304, 33.31036155896686, -28.838912135698926, -1.4693369446360407,
    -10.418532078512769, 73.36998380030019, -21.21493655887511, 44.00620956459555, 53.63261051536182,
    2.7285937751837395, 13.470863898732262, 18.406813289986204, 79.81576486014623, 11.44319844208276,
    54.93447134455431, 57.30432717171027, 40.23921236138773, 65.67885067903487, 26.366481706628406,
    -27.30759152512293, 25.811669049736366, 33.765769465674026, 46.064188205488364, 68.68320146596031,
    -30.14247692338735, -20.27099815222924, 72.28995446564878, -23.225596585148693, -17.707777462132427,
    -16.533526290183076, 52.71601762751186, -34.19215602995044, 17.794220792755056, 62.28063781336567,
    48.17997449949212, -62.87666397080855, -0.3946724203053975, -63.61229513391514, 62.025414757794124,
    -53.919013466682095, 22.71650382909786, -48.50198607380415, 39.16369296912181, -33.30665184747603,
    69.78065114460998, 44.39743053916996, -79.65007815070732, 4.313169443398394, -38.431776573060716,
    -3.2886541591535376, -4.464771381506893, -11.851966954379932, 53.314696580174, -47.80425745244058,
    -28.361356211321436, 46.48058986649623, 24.913829708880712, 32.701013472334154, -16.81491036111501,
    -74.32158198349097, 62.781854475534715, -39.33464299745868, 71.914280371025, 43.45306457719519,
    45.34911724499962, 17.635557921803695, 42.528929208317535, 57.38289380412539, -59.172599789657966,
    68.02685304924859, -22.83947203810746, 61.909059219612146, -25.281987529689864, -39.32991901928908,
    21.52408688800091, 19.141746734135268, 33.404708887073156, -40.93749669488551, -24.985279381108313
])

shift_sum_of_squares = np.array([
    22.479705250884777, 60.75750080751871, -59.15674324461888, 27.56336400934657, -59.29279403989607,
    -64.68447838216555, 78.33759532500437, -76.51052473538245, -66.6507114149451, 64.93462028133038,
    -15.23828302367869, -17.825823787619903, -31.92405019180454, 67.72646439056987, -0.9830868197666973,
    39.91084384292476, 1.4187967568760484, 24.815734535933146, -3.3313494842577285, 18.8446220852682,
    -17.002497910454828, 50.326561978450655, 53.95001535109091, 18.403170606945253, -21.184841218701195,
    -62.069991345971154, -43.868924783714164, -24.682337470629072, -2.8605529514137373, 65.39162073164252,
    2.6654994832190937, 23.60580815931415, 13.846698992449433, -41.45381813820245, -76.7118669691122,
    40.566665904615746, 22.28454884887448, -55.31708080994516, -76.649068909635, 40.966439161432476,
    -3.0455255815577402, 3.119008384655956, 18.342768255788243, -20.144695942649946, 8.54936385804288,
    50.45305033701291, 23.2263485421784, 10.68869991058412, 74.77290130427247, 33.82014641772588,
    -19.74543928622861, -43.37360584484172, 79.02166733921979, -45.92410252364756, -44.01344981695157,
    44.109743914203506, -77.20689956137252, -72.65296438262618, 46.613878010132, -14.343495951611033,
    25.970325670723298, -2.355868084672778, -72.92020917892613, 68.27133428368728, 60.8093904760039,
    44.33884079839375, 15.978963362849413, 6.74177622913723, 54.68618170146863, 75.90821354473843,
    32.160082912969955, 55.10526379835744, -20.089763211464202, -63.30186719081464, -29.970046051685806,
    -59.730237380251964, -74.06534479168236, -7.848712706118931, -51.60763845461748, -47.8986829918213,
    -46.24410011332074, 68.54056330770274, 41.40086049990089, -45.113643245610234, -53.385722169041614,
    -28.50002393256048, -9.680142709402858, 79.96711109340981, 78.51165296114377, 54.834496959318756,
    -64.63034566544378, -21.84107572223855, 48.37382176099271, -51.04930456902558, -43.1167085334479,
    40.877819320236654, 21.594957137775253, -12.021210965770223, 51.08608568221766, 13.845330603293263
])

shift_schwefel = np.array([
    65.90306349157711, 7.223609380925922, 48.06403595428989, -61.23008904674027, -56.996531732952235,
    -65.67006959079087, 56.81852866354663, -49.51353093024842, -42.403457350576694, 64.22460834371887,
    -15.782490031817943, 78.2275393766464, 58.981759831549084, -41.74229801452056, 70.70390026719807,
    -27.120714671264494, -43.362534341811525, -11.357893018802812, 25.329639322623194, -24.02213869456814,
    -29.51839742945309, -50.65352492132828, -17.33907934005014, -75.7276769694174, 25.60834507308691,
    -40.61133175959546, 53.35222265494423, 30.414516938139826, -37.83651132063628, 30.243572616044418,
    -2.2080543227080227, -51.34193306616531, 37.36039082956739, -44.93167877354562, 63.19570228206453,
    -0.3230550173947364, 54.997116479336086, -39.773290218583256, 54.1565458883382, 12.124637138785076,
    43.85247188324382, -75.91826727683562, 49.74255951193999, 63.92380934836413, 54.34684698334098,
    -19.563396930885908, 31.656857648987852, -32.907795698796306, -29.963471074339374, -60.672896416251675,
    -77.62660870400686, -75.95478227792732, 23.6157226667015, 75.57747129002121, -15.915369172179027,
    -45.240636434786694, 34.3519391836095, 65.07767014581174, 13.32242757194949, 71.34237817983137,
    -74.51968139294803, 3.512587033090128, 56.30141471139396, 31.538703365256055, 24.585815604555407,
    -67.48845399940869, -40.64252673369548, 59.91338156677921, -74.95426285566866, -38.150045876281204,
    -43.153328841782056, 21.868730315180528, -41.62610523688311, 1.6077581521206525, -72.85317012180792,
    63.05034100312395, 48.208504063943536, -19.726627265139726, -14.832261021455807, -5.053624253018285,
    -60.42559524043723, -53.2642183230471, -48.14019430704212, 35.20030522956961, 79.05287296526038,
    -79.9325666407405, 59.03982443403498, 53.3011236362469, 20.874500877568096, 44.9656312058855,
    74.28787167461365, 44.31739930419482, -78.40723117562789, -54.82586056439315, -36.33222788844623,
    69.06988428571137, 79.06863059997934, 22.41510141643954, 61.96066562216663, -5.6050556467548205
])

shift_rastrigin = np.array([
    4.070088073870897, -3.3584914395112166, 2.5276924556298725, 1.373781594176445, -1.8387129521132595,
    2.057549949794417, -0.0023364913111585395, 2.013487904123477, -3.7248964059839147, -0.33919629789907413,
    2.8301821463273367, -2.2929959706368352, -2.2706325455604484, -3.513692538288365, -1.523336663056385,
    -4.0764777207287075, -1.6237586256460155, 0.9178097684770039, -0.2723487256238868, -2.9841270159498463,
    2.9288432571682828, 0.22934413055341984, 3.720378352924264, -1.429905611686805, 1.6068438224463835,
    2.1513735804078467, -0.6971225700819095, -2.8180867567781878, -1.2171333029086893, -4.064563849878518,
    1.3543350787859474, 2.4050645503835417, -2.5584324886855985, 1.5660383241939355, 2.1289004812453385,
    3.8337296275949706, -0.5606593824135673, 1.141577243686811, 0.8120054159429309, 0.5394159295625087,
    -2.5598649074502093, 4.033201745873486, 3.24227861085845, 2.916933432390337, 3.639752419533484,
    1.5188619717725969, 3.839178330380485, 3.975865667365813, -1.191858979126291, -3.489747339689812,
    3.6319133591436685, -0.8694397826397151, -0.039238131176527524, 0.857258173067124, 3.5742721242496724,
    -1.2964486629139182, 0.622681353542129, -3.2417846861744932, -3.5847655715322952, -0.20674408185538518,
    -1.4176918157719318, -1.8093890874563168, -2.4458826348346667, -2.4951135809966636, -0.3701415900112224,
    -0.9412743172996478, -2.5093000736061777, -0.957089975451122, -0.34490389779906305, -0.9080868663263582,
    2.5324398853872463, 0.7681006679604776, 0.0686178202466019, 3.3249963877119377, -1.7808876498791788,
    -2.765311233368114, -4.021295100468257, -3.0287066679231973, 1.1237366969449676, 2.593350377277937,
    -2.882030611770738, -1.0382816516673712, -2.7461834729291623, -3.886857036748543, 2.3862007049941916,
    1.7060737084162199, 3.0561050423140896, 2.7412547729688193, 3.6348483498236694, 2.0446951025524065,
    -2.1226445447582245, 2.2055941964197308, -1.881997196954174, -2.0742308641271237, -4.0625624578635255,
    -3.0987340790486426, 2.5444665506465656, -1.6879107868549954, -2.299820194854612, -0.6872363415421097
])

shift_ackley = np.array([
    21.125524221556333, -11.11861062406522, -8.384759722218377, -4.998410532366073, 15.080716791129063,
    17.219200462474568, 17.30816180946608, 10.917710600242835, 20.963400683396372, 19.33124762324178,
    -10.07785283476705, -0.6470251048379687, -9.17403575481461, 0.2263125149343992, 3.5360559029458436,
    -24.976221736741277, 5.4509130845083185, -22.008998257857435, 4.319939429353628, -0.015227633728962076,
    -10.266583117363774, -2.984262944040246, -14.575704508621596, 9.332038871399178, 12.260212736282845,
    2.6286817951067007, 3.7062873124753715, -7.422751349625646, 22.839645124520274, 18.139957746273467,
    -23.12538796733297, 3.106978031277709, -21.8650793129579, 23.285704349592457, -8.877654886311145,
    -15.936478675865914, -14.529257223964459, -0.5256354355698463, 3.421693809849465, -22.925462999946205,
    -13.789171558857868, -12.260934479396866, 0.8518120279262078, -6.2876202233673375, 17.750101847964885,
    24.91013310009094, -3.1246259173628452, -7.631501678844927, -7.124798758640981, 0.9420940535559126,
    -3.5298708787570163, -13.655076475547936, -3.220475098055129, -0.7478333903264627, 13.62921655558997,
    -3.336289145576181, -1.7729883120516092, -2.487660518823951, 13.831869546225832, 3.8314346375240227,
    -17.39761276709943, 10.865095694434025, 15.228433667840704, -16.627545155754365, -20.431272665369136,
    -6.458592974832296, 7.631660014538554, 3.074842377520376, -10.126681052067994, 5.3638689257400145,
    21.993909890455164, 9.497579963476142, -25.38346714493473, 23.309384518115635, 3.51234224076034,
    22.789117409253414, 17.335098841147875, 16.853847369236334, -22.711831662040623, -21.43493872394238,
    14.12580762325981, 12.670980727215792, -0.18261513276357988, -2.756069406807832, -13.440337358321415,
    -7.500233746130899, -22.829280727381548, 22.39647155450566, -9.793500872001813, -11.480708343814184,
    -11.497396176336123, -17.84442861516022, 16.259172390942574, -4.7388363410613366, -22.313184156325793,
    -16.909573209790985, 21.735083164944925, -13.918617730409967, -21.395853820675416, -22.481403053600452
])

shift_griewank = np.array([
    104.94691081342978, -135.6761348470227, 92.81100891955543, -10.87108518188711, 407.63013054172563,
    -223.31315231746964, -224.56627659185563, -222.7632371161696, -338.7309010063917, -236.65660889064418,
    -102.44810336184622, 126.42998956660733, 145.51932139597716, 362.4195007946686, -136.07822974158148,
    385.3787391034516, 224.54916653990915, 43.478124239142176, 174.10299590196337, -214.8310656032395,
    -398.01422639093084, -309.2614310193993, -178.06570153291875, -162.6651823263993, 450.850657541738,
    -454.92932203096916, -419.61768171271166, -297.80826342133184, 66.9431726795051, 81.54612611060406,
    33.75350391540678, 452.7503721088576, -394.37679317004665, 211.96497077719948, 209.2762825707889,
    -456.2796841185877, 43.786866583660526, 39.94139518856173, -392.0757065399322, 368.3834199650955,
    -230.1695108177757, 260.9386598025741, 424.9982235544909, 219.5539088594145, -56.74195565523951,
    353.31823225966764, -314.409074063865, -344.22129416136335, 350.389300480917, 210.6305580593255,
    -351.3688705488921, -419.56080494622336, -352.4149162964608, -306.0148711883166, 234.1092941008617,
    179.7062426106279, -180.45299833941465, -353.7895714765393, 304.63535113945875, 11.040575864524044,
    409.40525097198395, -417.0405885951272, 460.81384815623824, 476.25681692986484, 248.03325986310347,
    126.13612920930825, 366.9901574947629, 40.28445536257732, 196.48680138749523, -89.56987522823874,
    -31.05376733595574, -254.8501756921979, 341.9328159488531, -366.2937316074139, -98.73389669853748,
    -318.98905799691175, -152.38232167124994, 367.26470889921757, -72.94267032929105, 372.16601829879653,
    96.98492379728134, -385.9422293412532, -94.28273010579477, 346.5708159613322, -139.0701306975041,
    -62.71021784084701, -477.67009305148014, 267.4589914695547, 135.44409046482406, -107.786912911326,
    12.944227804931472, 452.8144059395564, -2.2036047536961973, -458.0828323445677, -305.00083158848463,
    352.90943721979033, -173.23435776482415, -294.46059074937193, 329.82221519045083, 295.09636446279023
])

# Problem definitions
def shifted_sphere(x, shift=None, n=None):
    if shift is None:
        shift = np.random.uniform(-100, 100, n)
    return cma.ff.sphere(x - shift)

def sum_of_squares(x):
    fitness = 0
    for i in range(len(x)):
        fitness += (i + 1) * (x[i] ** 2)
    return fitness

def shifted_sum_of_squares(x, shift=None, n=None):
    if shift is None:
        shift = np.random.uniform(-100, 100, n)
    return sum_of_squares(x - shift)

def schwefel(x):
    fitness = 0
    for i in range(len(x)):
        sum_j = sum(x[i] for j in range(i))  # Sum up to i
        fitness += sum_j ** 2
    return fitness

def shifted_schwefel(x, shift=None, n=None):
    if shift is None:
        shift = np.random.uniform(-100, 100, n)
    return schwefel(x - shift)

def shifted_rastrigin(x, shift=None, n=None):
    if shift is None:
        shift = np.random.uniform(-100, 100, n)
    return cma.ff.rastrigin(x - shift)

def ackley(x):
    a = 20.0
    b = 0.2
    c = 2 * np.pi
    n = len(x)
    sum1 = sum(x[i] ** 2 for i in range(n))
    sum2 = sum(np.cos(c * x[i]) for i in range(n))
    return -a * np.exp(-b * np.sqrt(sum1 / n)) - np.exp(sum2 / n) + a + np.e

def shifted_ackley(x, shift=None, n=None):
    if shift is None:
        shift = np.random.uniform(-100, 100, n)
    return ackley(x - shift)

def griewank(x):
    sum_ = 0
    prod = 1
    for i in range(len(x)):
        sum_ += x[i] * x[i]
        prod *= np.cos(x[i] / np.sqrt(i + 1))
    return (1.0 / 4000.0) * sum_ - prod + 1

def shifted_griewank(x, shift=None, n=None):
    if shift is None:
        shift = np.random.uniform(-100, 100, n)
    return griewank(x - shift)

def rosenbrock(x):
    fitness = 0
    for i in range(len(x) - 1):
        fitness += 100 * (x[i + 1] - x[i] * x[i]) ** 2 + (1 - x[i]) ** 2
    return fitness

def shekels_foxholes(x):
    x = np.asarray(x, dtype=np.float64)  # Ensure floating-point input
    a = np.array([
        [-32, -32], [-16, -32], [0, -32], [16, -32], [32, -32],
        [-32, -16], [-16, -16], [0, -16], [16, -16], [32, -16],
        [-32, 0], [-16, 0], [0, 0], [16, 0], [32, 0],
        [-32, 16], [-16, 16], [0, 16], [16, 16], [32, 16],
        [-32, 32], [-16, 32], [0, 32], [16, 32], [32, 32]
    ])
    fitness = 0
    for j in range(25):
        sum_j = sum((x[i] - a[j, i]) ** 6 for i in range(len(x))) + (j + 1)
        fitness += 1.0 / sum_j
    fitness += 1.0 / 500.0
    return 1.0 / fitness

def six_hump_camel_back(x):
    return (4 * x[0] ** 2
            - 2.1 * x[0] ** 4
            + (1.0 / 3.0) * x[0] ** 6
            + x[0] * x[1]
            - 4 * x[1] ** 2
            + 4 * x[1] ** 4)

def branin(x):
    return ((x[1] - (5.1 / (4 * np.pi * np.pi)) * x[0] * x[0] + (5.0 / np.pi) * x[0] - 6) ** 2
            + 10 * (1 - 1.0 / (8.0 * np.pi)) * np.cos(x[0]) + 10)

def goldstein_price(x):
    return ((1 + (x[0] + x[1] + 1) ** 2 * (19 - 14 * x[0] + 3 * x[0] * x[0] - 14 * x[1] + 6 * x[0] * x[1] + 3 * x[1] * x[1])) *
            (30 + (2 * x[0] - 3 * x[1]) ** 2 * (18 - 32 * x[0] + 12 * x[0] * x[0] + 48 * x[1] - 36 * x[0] * x[1] + 27 * x[1] * x[1])))

def hartman(x):
    c = np.array([1.0, 1.2, 3.0, 3.2])
    a = np.array([[3.0, 10, 30], [0.1, 10, 35], [3.0, 10, 30], [0.1, 10, 35]])
    p = np.array([[0.3689, 0.1170, 0.2673], [0.4699, 0.4387, 0.7470], [0.1091, 0.8732, 0.5547], [0.03815, 0.5743, 0.8828]])
    fitness = 0
    for i in range(4):
        sum_j = sum(a[i][j] * (x[j] - p[i][j]) ** 2 for j in range(len(x)))
        fitness += c[i] * np.exp(-sum_j)
    return -fitness

# Function to run CMA-ES for a problem
def run_cmaes_for_problem(options, fun, problem_name, n_dimensions, bounds, sigma0, number_of_runs):

    results = []
    for i in range(number_of_runs):
        print(f"Run: {i + 1} for {problem_name}")
        x0 = np.random.uniform(bounds[0], bounds[1], n_dimensions)
        xopt, es = cma.fmin2(fun, x0, sigma0, options)
        results.append(es.result.fbest)
    
    # Write results to file
    project_dir = Path.cwd()
    parent_dir = project_dir.parent.parent.parent
    files_dir = parent_dir / "EARS comparison" / "Algorithm results"
    file_location = files_dir / f"CMA-ES-Author-Python_{problem_name}D{n_dimensions}.txt"
    files_dir.mkdir(parents=True, exist_ok=True)
    with open(file_location, 'w') as f:
        for result in results:
            f.write(f"{result}\n")
    
    print(f"Best fitness values for {problem_name} saved to {file_location}")
    print(f"Mean fitness: {np.mean(results):.4e}")
    print(f"Min fitness: {np.min(results):.4e}")
    print(f"Max fitness: {np.max(results):.4e}")

# List of problems
problems = [
    {'fun': lambda x: shifted_sphere(x, shift=shift_sphere[:n_dimensions]), 'name': 'ShiftedSphere', 'n_dimensions': n_dimensions, 'bounds': [[-100]*n_dimensions, [100]*n_dimensions]},
    {'fun': lambda x: shifted_sum_of_squares(x, shift=shift_sum_of_squares[:n_dimensions]), 'name': 'ShiftedSumOfSquares', 'n_dimensions': n_dimensions, 'bounds': [[-100]*n_dimensions, [100]*n_dimensions]},
    {'fun': lambda x: shifted_schwefel(x, shift=shift_schwefel[:n_dimensions]), 'name': 'ShiftedSchwefel', 'n_dimensions': n_dimensions, 'bounds': [[-100]*n_dimensions, [100]*n_dimensions]},
    {'fun': lambda x: shifted_rastrigin(x, shift=shift_rastrigin[:n_dimensions]), 'name': 'ShiftedRastrigin', 'n_dimensions': n_dimensions, 'bounds': [[-5.12]*n_dimensions, [5.12]*n_dimensions]},
    {'fun': lambda x: shifted_ackley(x, shift=shift_ackley[:n_dimensions]), 'name': 'ShiftedAckley', 'n_dimensions': n_dimensions, 'bounds': [[-32]*n_dimensions, [32]*n_dimensions]},
    {'fun': lambda x: shifted_griewank(x, shift=shift_griewank[:n_dimensions]), 'name': 'ShiftedGriewank', 'n_dimensions': n_dimensions, 'bounds': [[-600]*n_dimensions, [600]*n_dimensions]},
    {'fun': cma.ff.sphere, 'name': 'Sphere', 'n_dimensions': n_dimensions, 'bounds': [[-100]*n_dimensions, [100]*n_dimensions]},
    {'fun': sum_of_squares, 'name': 'SumOfSquares', 'n_dimensions': n_dimensions, 'bounds': [[-100]*n_dimensions, [100]*n_dimensions]},
    {'fun': schwefel, 'name': 'Schwefel', 'n_dimensions': n_dimensions, 'bounds': [[-100]*n_dimensions, [100]*n_dimensions]},
    {'fun': cma.ff.rastrigin, 'name': 'Rastrigin', 'n_dimensions': n_dimensions, 'bounds': [[-5.12]*n_dimensions, [5.12]*n_dimensions]},
    {'fun': ackley, 'name': 'Ackley', 'n_dimensions': n_dimensions, 'bounds': [[-32]*n_dimensions, [32]*n_dimensions]},
    {'fun': griewank, 'name': 'Griewank', 'n_dimensions': n_dimensions, 'bounds': [[-600]*n_dimensions, [600]*n_dimensions]},
    {'fun': rosenbrock, 'name': 'Rosenbrock', 'n_dimensions': n_dimensions, 'bounds': [[-30]*n_dimensions, [30]*n_dimensions]},
    {'fun': shekels_foxholes, 'name': 'ShekelsFoxholes', 'n_dimensions': 2, 'bounds': [[-65.536, -65.536], [65.536, 65.536]]},
    {'fun': six_hump_camel_back, 'name': 'SixHumpCamelBack', 'n_dimensions': 2, 'bounds': [[-5, -5], [5, 5]]},
    {'fun': branin, 'name': 'Branin', 'n_dimensions': 2, 'bounds': [[-5, 0], [10, 15]]},
    {'fun': goldstein_price, 'name': 'GoldsteinPrice', 'n_dimensions': 2, 'bounds': [[-2, -2], [2, 2]]},
    {'fun': hartman, 'name': 'Hartman', 'n_dimensions': 3, 'bounds': [[0]*3, [1]*3]}
]
start_time = time.perf_counter()
# Run CMA-ES for each problem
for problem in problems:
    problem_name = problem['name']
    options = {
        'popsize': pop_size,
        'bounds': problem['bounds'],
        'maxfevals': max_evaluations,
        'tolfun': 1e-8,
        'verb_disp': 0,
        'verb_filenameprefix': '',  # Disable file output
    }
        
    run_cmaes_for_problem(
        options = options,
        fun=problem['fun'],
        problem_name=problem['name'],
        n_dimensions=problem['n_dimensions'],
        bounds=problem['bounds'],
        sigma0=sigma0,
        number_of_runs = number_of_runs
    )
end_time = time.perf_counter()
runtime = end_time - start_time
print(f"Runtime: {runtime:.6f} seconds")

Run: 1 for ShiftedSphere
Run: 2 for ShiftedSphere
Run: 3 for ShiftedSphere
Run: 4 for ShiftedSphere
Run: 5 for ShiftedSphere
Run: 6 for ShiftedSphere
Run: 7 for ShiftedSphere
Run: 8 for ShiftedSphere
Run: 9 for ShiftedSphere
Run: 10 for ShiftedSphere
Run: 11 for ShiftedSphere
Run: 12 for ShiftedSphere
Run: 13 for ShiftedSphere
Run: 14 for ShiftedSphere
Run: 15 for ShiftedSphere
Run: 16 for ShiftedSphere
Run: 17 for ShiftedSphere
Run: 18 for ShiftedSphere
Run: 19 for ShiftedSphere
Run: 20 for ShiftedSphere
Run: 21 for ShiftedSphere
Run: 22 for ShiftedSphere
Run: 23 for ShiftedSphere
Run: 24 for ShiftedSphere
Run: 25 for ShiftedSphere
Run: 26 for ShiftedSphere
Run: 27 for ShiftedSphere
Run: 28 for ShiftedSphere
Run: 29 for ShiftedSphere
Run: 30 for ShiftedSphere
Run: 31 for ShiftedSphere
Run: 32 for ShiftedSphere
Run: 33 for ShiftedSphere
Run: 34 for ShiftedSphere
Run: 35 for ShiftedSphere
Run: 36 for ShiftedSphere
Run: 37 for ShiftedSphere
Run: 38 for ShiftedSphere
Run: 39 for ShiftedSp

D:\Repositories\Optimization-frameworks-comparison\Authors source code\CMA-ES\cma\utilities\utils.py:351: UserWarning: sigma change np.exp(1.0180947672547795) = 2.7679162125223695 clipped to np.exp(+-1) (time=May  9 18:52:05 2025 class=CMAAdaptSigmaCSA method=update iteration=4)
  ')', **kwargs_for_warn)


Run: 9 for ShekelsFoxholes
Run: 10 for ShekelsFoxholes


D:\Repositories\Optimization-frameworks-comparison\Authors source code\CMA-ES\cma\utilities\utils.py:351: UserWarning: sigma change np.exp(1.072606882939924) = 2.922989467896987 clipped to np.exp(+-1) (time=May  9 18:52:06 2025 class=CMAAdaptSigmaCSA method=update iteration=4)
  ')', **kwargs_for_warn)


Run: 11 for ShekelsFoxholes
Run: 12 for ShekelsFoxholes
Run: 13 for ShekelsFoxholes
Run: 14 for ShekelsFoxholes


...

Run: 95 for ShekelsFoxholes
Run: 96 for ShekelsFoxholes
Run: 97 for ShekelsFoxholes
Run: 98 for ShekelsFoxholes
Run: 99 for ShekelsFoxholes
Run: 100 for ShekelsFoxholes
Best fitness values for ShekelsFoxholes saved to D:\Repositories\Optimization-frameworks-comparison\EARS comparison\Algorithm results\CMA-ES-Author-Python_ShekelsFoxholesD2.txt
Mean fitness: 9.7090e+00
Min fitness: 9.9800e-01
Max fitness: 2.3809e+01
Run: 1 for SixHumpCamelBack
Run: 2 for SixHumpCamelBack
Run: 3 for SixHumpCamelBack
Run: 4 for SixHumpCamelBack
Run: 5 for SixHumpCamelBack
Run: 6 for SixHumpCamelBack
Run: 7 for SixHumpCamelBack
Run: 8 for SixHumpCamelBack
Run: 9 for SixHumpCamelBack
Run: 10 for SixHumpCamelBack
Run: 11 for SixHumpCamelBack
Run: 12 for SixHumpCamelBack


D:\Repositories\Optimization-frameworks-comparison\Authors source code\CMA-ES\cma\utilities\utils.py:351: UserWarning: sigma change np.exp(1.1182883250386986) = 3.059612656285309 clipped to np.exp(+-1) (time=May  9 18:52:32 2025 class=CMAAdaptSigmaCSA method=update iteration=3)
  ')', **kwargs_for_warn)


Run: 13 for SixHumpCamelBack
Run: 14 for SixHumpCamelBack
Run: 15 for SixHumpCamelBack
Run: 16 for SixHumpCamelBack
Run: 17 for SixHumpCamelBack
Run: 18 for SixHumpCamelBack
Run: 19 for SixHumpCamelBack
Run: 20 for SixHumpCamelBack
Run: 21 for SixHumpCamelBack
Run: 22 for SixHumpCamelBack
Run: 23 for SixHumpCamelBack
Run: 24 for SixHumpCamelBack
Run: 25 for SixHumpCamelBack
Run: 26 for SixHumpCamelBack
Run: 27 for SixHumpCamelBack
Run: 28 for SixHumpCamelBack
Run: 29 for SixHumpCamelBack
Run: 30 for SixHumpCamelBack
Run: 31 for SixHumpCamelBack
Run: 32 for SixHumpCamelBack
Run: 33 for SixHumpCamelBack
Run: 34 for SixHumpCamelBack
Run: 35 for SixHumpCamelBack
Run: 36 for SixHumpCamelBack
Run: 37 for SixHumpCamelBack
Run: 38 for SixHumpCamelBack
Run: 39 for SixHumpCamelBack
Run: 40 for SixHumpCamelBack
Run: 41 for SixHumpCamelBack
Run: 42 for SixHumpCamelBack
Run: 43 for SixHumpCamelBack
Run: 44 for SixHumpCamelBack
Run: 45 for SixHumpCamelBack
Run: 46 for SixHumpCamelBack
Run: 47 for Si

D:\Repositories\Optimization-frameworks-comparison\Authors source code\CMA-ES\cma\utilities\utils.py:351: UserWarning: sigma change np.exp(1.011727284812826) = 2.7503475480793593 clipped to np.exp(+-1) (time=May  9 18:52:56 2025 class=CMAAdaptSigmaCSA method=update iteration=2)
  ')', **kwargs_for_warn)


Run: 39 for Branin
Run: 40 for Branin
Run: 41 for Branin
Run: 42 for Branin
Run: 43 for Branin
Run: 44 for Branin
Run: 45 for Branin
Run: 46 for Branin
Run: 47 for Branin
Run: 48 for Branin
Run: 49 for Branin
Run: 50 for Branin
Run: 51 for Branin
Run: 52 for Branin
Run: 53 for Branin
Run: 54 for Branin
Run: 55 for Branin
Run: 56 for Branin
Run: 57 for Branin
Run: 58 for Branin
Run: 59 for Branin
Run: 60 for Branin
Run: 61 for Branin
Run: 62 for Branin
Run: 63 for Branin
Run: 64 for Branin
Run: 65 for Branin
Run: 66 for Branin
Run: 67 for Branin
Run: 68 for Branin
Run: 69 for Branin
Run: 70 for Branin
Run: 71 for Branin
Run: 72 for Branin
Run: 73 for Branin
Run: 74 for Branin
Run: 75 for Branin
Run: 76 for Branin
Run: 77 for Branin
Run: 78 for Branin
Run: 79 for Branin
Run: 80 for Branin
Run: 81 for Branin
Run: 82 for Branin
Run: 83 for Branin
Run: 84 for Branin
Run: 85 for Branin
Run: 86 for Branin
Run: 87 for Branin
Run: 88 for Branin
Run: 89 for Branin
Run: 90 for Branin
Run: 91 for 